In [10]:
import torch
import numpy as np
from torch import nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn.init

In [11]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(777)

In [23]:
learning_rate = 0.001
training_epochs= 100
batch_size = 256

In [24]:
mnist_train = dsets.MNIST(root='MNIST_data',
    train=True,
    transform=transforms.ToTensor(),
    download=True
)
mnist_test = dsets.MNIST(root='MNIST_data',
    train=False,
    transform=transforms.ToTensor(),
    download=True
)

In [25]:
mnist_train

Dataset MNIST
    Number of datapoints: 60000
    Root location: MNIST_data
    Split: Train
    StandardTransform
Transform: ToTensor()

In [26]:
data_loader = torch.utils.data.DataLoader(
    dataset=mnist_train,
    batch_size = batch_size,
    shuffle=True,
    drop_last = True
)

In [27]:
data_loader[1]

TypeError: 'DataLoader' object is not subscriptable

In [28]:
mnist_test

Dataset MNIST
    Number of datapoints: 10000
    Root location: MNIST_data
    Split: Test
    StandardTransform
Transform: ToTensor()

In [29]:
# class cnn(nn.Module):
#     def __init__(self):
#         super().__init__()
#         self.keep_prob = 0.5
#         self.layer1 = torch.nn.Sequential(
#             torch.nn.Conv2d(1,16,3,1,padding=1),
#             torch.nn.ReLU(),
#             torch.nn.MaxPool2d(2,stride=1)
#         )
#         self.layer2 = torch.nn.Sequential(
#             torch.nn.Conv2d(16,32,3,1,1),
#             torch.nn.ReLU(),
#             torch.nn.MaxPool2d(2, stride=1)
#         )
#         self.layer3 = torch.nn.Sequential(
#             torch.nn.Conv2d(32,32,3,1,1),
#             torch.nn.ReLU(),
#             torch.nn.MaxPool2d(2,stride=1)
#         )
#         self.fc1 = torch.nn.Linear(4 * 4 * 8, 10, bias = True)
#         # torch.nn.init.xavier_uniform_(self.fc1.weight)
#         # self.layer4 = torch.nn.Sequential(
#         #     self.fc1,
#         #     torch.nn.SiLU(),
#         #     torch.nn.Dropout( p = 1 - self.keep_prob)
#         # )
#         # self.fc2 = torch.nn.Linear(625,10,bias = True)
#         # torch.nn.init.xavier_uniform_(self.fc2.weight)
    
#     def forward(self, x):
#         out = self.layer1(x)
#         out = self.layer2(out)
#         out = self.layer3(out)
#         # out = out.view(out.size(0), -1)
#         # out = self.layer4(out)
#         out = self.fc1(out)
#         return out


class cnn(nn.Module):
    def __init__(self):
        super().__init__()
        self.keep_prob = 0.5
        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(1,32,3,1,padding=1),
            torch.nn.SiLU(),
            torch.nn.MaxPool2d(2,stride=1)
        )
        self.layer2 = torch.nn.Sequential(
            torch.nn.Conv2d(32,64,3,1,1),
            torch.nn.SiLU(),
            torch.nn.MaxPool2d(2, stride=1)
        )
        self.layer3 = torch.nn.Sequential(
            torch.nn.Conv2d(64,128,3,1,1),
            torch.nn.SiLU(),
            torch.nn.MaxPool2d(2,2,1)
        )
        # self.fc1 = torch.nn.Linear(4 * 4 * 32, 10, bias = True)
        self.fc1 = torch.nn.Linear(25088, 10, bias = True)
        # torch.nn.init.xavier_uniform_(self.fc1.weight)
        # self.layer4 = torch.nn.Sequential(
        #     self.fc1,
        #     torch.nn.SiLU(),
        #     torch.nn.Dropout( p = 1 - self.keep_prob)
        # )
        # self.fc2 = torch.nn.Linear(625,10,bias = True)
        # torch.nn.init.xavier_uniform_(self.fc2.weight)
    
    def forward(self, x):
        # print(x.shape)
        out = self.layer1(x)
        # print(out.shape)
        out = self.layer2(out)
        # print(out.shape)
        out = self.layer3(out)
        # print(out.shape)
        out = out.view(out.size(0), -1)
        # print(out.shape)
        # out = self.layer4(out)
        out = self.fc1(out)
        return out

In [30]:
model = cnn().to(device)

In [31]:
criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr = 0.01)

In [32]:

total_batch = len(data_loader)

In [33]:
for epoch in range(training_epochs):
    avg_cost = 0

    for X, Y in data_loader:
        X = X.to(device)
        Y = Y.to(device)

        optimizer.zero_grad()
        hypothesis = model(X)
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()

        avg_cost += cost/total_batch

    print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch + 1 , avg_cost))
    

[Epoch:    1] cost = 1.7196846
[Epoch:    2] cost = 0.0964167342
[Epoch:    3] cost = 0.0665245429
[Epoch:    4] cost = 0.0572907552
[Epoch:    5] cost = 0.0520264506
[Epoch:    6] cost = 0.0450050496
[Epoch:    7] cost = 0.0411917195
[Epoch:    8] cost = 0.043069765
[Epoch:    9] cost = 0.0476047732
[Epoch:   10] cost = 0.0435550772
[Epoch:   11] cost = 0.0396808945
[Epoch:   12] cost = 0.052599676
[Epoch:   13] cost = 0.046072457
[Epoch:   14] cost = 0.062129572
[Epoch:   15] cost = 0.0507975966
[Epoch:   16] cost = 0.0505004115
[Epoch:   17] cost = 0.0669750571
[Epoch:   18] cost = 0.140460655
[Epoch:   19] cost = 1.0090791
[Epoch:   20] cost = 0.143159613
[Epoch:   21] cost = 0.095900014
[Epoch:   22] cost = 0.0712244064
[Epoch:   23] cost = 0.0578233749
[Epoch:   24] cost = 0.0610677227
[Epoch:   25] cost = 0.0496310554
[Epoch:   26] cost = 0.0429023765
[Epoch:   27] cost = 0.0428903103
[Epoch:   28] cost = 0.0379214436
[Epoch:   29] cost = 0.0350372382
[Epoch:   30] cost = 0.0308